In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
s1 = pd.read_csv('s1_data.csv')
s2 = pd.read_csv('s2_data.csv')

In [3]:
# sort s1 and s2 by date 
s1 = s1.sort_values(by=['date'])
s2 = s2.sort_values(by=['date'])

In [4]:
s1.head()

,geometry,label,date,tile_number,id,location_id,image_dir
131,"[[[30.033794, -1.947753], [30.033877, -1.99400...",0,2018-12-15,25,sen12floods_s1_labels_0025_2018_12_15,25,sen12flood/sen12floods_s1_source/sen12floods_s...
121,"[[[30.125653, -1.901331], [30.125737, -1.94758...",0,2018-12-15,24,sen12floods_s1_labels_0024_2018_12_15,24,sen12flood/sen12floods_s1_source/sen12floods_s...
141,"[[[30.079766, -1.947669], [30.079851, -1.99392...",0,2018-12-15,26,sen12floods_s1_labels_0026_2018_12_15,26,sen12flood/sen12floods_s1_source/sen12floods_s...
101,"[[[30.033712, -1.901496], [30.033794, -1.94775...",0,2018-12-15,22,sen12floods_s1_labels_0022_2018_12_15,22,sen12flood/sen12floods_s1_source/sen12floods_s...
111,"[[[30.079683, -1.901415], [30.079766, -1.94766...",0,2018-12-15,23,sen12floods_s1_labels_0023_2018_12_15,23,sen12flood/sen12floods_s1_source/sen12floods_s...


In [5]:
!mkdir TimewiseCSV

In [7]:
# for same date create csv files
base_path= "TimewiseCSV\\"
k={}
for i in range(len(s1)):
    date = s1['date'][i]
    k[date]=0
    k[date]+=1
    s1_date = s1[s1['date']==date]
    s2_date = s2[s2['date']==date]
    if(k[date]==1):
        s1_date.to_csv(base_path+str(date)+'_s1.csv',index=False)
        s2_date.to_csv(base_path+str(date)+'_s2.csv',index=False)


In [10]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
import pandas as pd
import os
import rasterio as rio
import cv2

def load_raster(filepath): # filepath of the raster file to be loaded
    '''load a single band raster'''
    with rio.open(filepath) as file: 
        # the squeeze method is called on the resulting array to remove any singleton dimensions 
        # (i.e., dimensions with size 1). This is done using the axis=0 argument, 
        # which tells squeeze to remove any singleton dimensions along the first axis.  
        raster = file.read().squeeze(axis=0)

        
    #we aregetting back the 2D image from singleton(1D)
    return raster

def load_s1_tiffs(folder,
                  scaling_values=[50.,100.]):
    images = []
    i = 0
    for im in sorted(os.listdir(folder)):
         
        if im.rsplit('.',maxsplit=1)[1] == 'tif':
            
            path = folder + '/' + im
            band = load_raster(path)
            band = band / scaling_values[i]
            
            band = cv2.resize(band,
                              CFG.img_size)
            
            images.append(band)
            i+=1 
                    
    return np.dstack(images)


def load_s2_tiffs(folder,
                  scaling_value=10000.):
    images = []
    for im in sorted(os.listdir(folder)):
        if im.rsplit('.',maxsplit=1)[1] == 'tif':    
            path = folder + '/' + im
            band = load_raster(path)
            band = band/ scaling_value
            
            band = cv2.resize(band,CFG.img_size)
            images.append(band)   

    return np.dstack(images)
                    
    
def tf_load_s1(path):    
    path = path.numpy().decode('utf-8')
    return load_s1_tiffs(path)
    
    

def tf_load_s2(path):    
    path = path.numpy().decode('utf-8')
    return load_s2_tiffs(path)

    
def process_image_s1(filename):
    '''function for preprocessing in tensorflow data'''
    
    return tf.py_function(tf_load_s1, 
                          [filename], 
                          tf.float32)



def process_image_s2(filename):
    '''function for preprocessing in tensorflow data'''
    
    return tf.py_function(tf_load_s2, 
                          [filename], 
                          tf.float32)



def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

class CFG:
    """
    This class provides a set of parameters and constants that may be used throughout a machine learning 
    pipeline for image classification, specifically in the context of identifying flooded areas.
    """
    seed = 3 # random initialization of weights in a machine learning model
    img_size = (256,256) # representing the dimensions of an image, specifically 256 x 256 pixels.
    BATCH_SIZE = 3 #  representing the number of samples that will be fed to a machine learning model during training.
    Autotune = tf.data.AUTOTUNE # a constant value from the tf.data.AUTOTUNE module that enables dynamic 
    # allocation of computational resources to improve performance.
    validation_size = 0.2 # a float value of 0.2 representing the fraction of the training dataset to be used for validation during training.
    class_dict= {0:'No Flooding', 
                 1: 'Flooding'}
    
    test_run = False # in training mode

def get_tf_dataset(image_paths,
                   labels=None, # put none for test data set
                   image_processing_fn=None,
                   augment_fn = None
                  ):
    
    
    '''returns a tf dataset object
    Inputs: 
    image_paths : paths to images
    labels: labels of each image
    image_processing_fn:  function to load and preprocess images 
    augment_fn : function to augment images '''
    
    #seperate datasets
    if labels is not None:
        labels_dataset = tf.data.Dataset.from_tensor_slices(labels)
    
    
    
    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    #load images 
    image_dataset = image_dataset.map(image_processing_fn,
                                      num_parallel_calls=tf.data.AUTOTUNE)
     
    if augment_fn is not None:
        
        image_dataset = image_dataset.map(augment_fn,
                                          num_parallel_calls=tf.data.AUTOTUNE)
     
    
    if labels is not None:
        return tf.data.Dataset.zip((image_dataset,labels_dataset))
    
    
    return image_dataset



def optimize_pipeline(tf_dataset,
                      batch_size = CFG.BATCH_SIZE,
                      Autotune_fn = CFG.Autotune,
                      cache= False,
                      batch = True):
    
    
    
    # prefetch(load the data with cpu,while gpu is training) the data in memory 
    tf_dataset = tf_dataset.prefetch(buffer_size=Autotune_fn)  
    if cache:
        tf_dataset = tf_dataset.cache()                        # store data in RAM  
        
    tf_dataset =  tf_dataset.shuffle(buffer_size=50)         # shuffle 
    
    if batch:
        tf_dataset = tf_dataset.batch(batch_size)              #split the data in batches  
    
    return tf_dataset

# Sentinel 1 dataset (not using augmentation here)


SAR_CNN = tf.keras.models.load_model('CNN_models/SAR_CNN.h5',
                                     custom_objects={'f1_score': f1_score,
                                                     'precision_m': precision_m,
                                                     'recall_m': recall_m})

def model_prediction(image):
    

    pred = np.argmax(SAR_CNN.predict(image[tf.newaxis,:,:,:]))
    # prd = int(pred.ravel())
    return pred

def set_data(s_data):
    S1_dataset = optimize_pipeline(tf_dataset=get_tf_dataset(image_paths = s_data.image_dir.values,
                                                labels = s_data.label,
                                                image_processing_fn = process_image_s1),
                                    
                                    batch_size = 3 * CFG.BATCH_SIZE)
    return S1_dataset

data= set_data(pd.read_csv('TimewiseCSV\\2018-12-16_s1.csv'))
for images,labels in data:
    # print(images.shape)
    # print(labels.shape)
    # print(labels)
    # print("IMG",images)
    for k in range(len(images)):
        # print("img",images[k])
        try:
            print(model_prediction(images[k]))
        except:
            pass


1/1 [==============================] - 1s 861ms/step
0
1/1 [==============================] - 0s 91ms/step
0
1/1 [==============================] - 0s 93ms/step
0
1/1 [==============================] - 0s 83ms/step
0
1/1 [==============================] - 0s 87ms/step
0
1/1 [==============================] - 0s 81ms/step
0
1/1 [==============================] - 0s 82ms/step
0
1/1 [==============================] - 0s 80ms/step
0
1/1 [==============================] - 0s 82ms/step
0
1/1 [==============================] - 0s 99ms/step
0
1/1 [==============================] - 0s 89ms/step
0


In [6]:
import numpy as np


# Load VV and VH polarization images
with rio.open('sen12flood\\sen12floods_s1_source\\sen12floods_s1_source\\sen12floods_s1_source_0286_2019_03_20\\VV.tif') as vv_ds:
    vv_img = vv_ds.read(1)
with rio.open('sen12flood\\sen12floods_s1_source\\sen12floods_s1_source\\sen12floods_s1_source_0286_2019_03_20\\VV.tif') as vh_ds:
    vh_img = vh_ds.read(1)

# Convert the images to float32 data type
vv_img = vv_img.astype(np.float32)
vh_img = vh_img.astype(np.float32)

# Apply a speckle filter using median filter
vv_img_filtered = rio.features.median_filter(vv_img, size=5)
vh_img_filtered = rio.features.median_filter(vh_img, size=5)

# Calculate the backscatter coefficient for VV and VH polarization
calibration_factor = 1.0
range_distance = 1.0
sigma_vv = 10.0 * np.log10(np.square(vv_img_filtered) / (calibration_factor * np.power(range_distance, 4)))
sigma_vh = 10.0 * np.log10(np.square(vh_img_filtered) / (calibration_factor * np.power(range_distance, 4)))

# Apply radiometric calibration
sigma_vv = sigma_vv - np.mean(sigma_vv)
sigma_vh = sigma_vh - np.mean(sigma_vh)

# Calculate the green, red, and near-infrared bands
green_band = sigma_vv / sigma_vh
red_band = sigma_vh / sigma_vv
nir_band = (sigma_vh - sigma_vv) / (sigma_vh + sigma_vv)

# Normalize the bands to 0-255 range
green_band_norm = np.uint8(255 * (green_band - green_band.min()) / (green_band.max() - green_band.min()))
red_band_norm = np.uint8(255 * (red_band - red_band.min()) / (red_band.max() - red_band.min()))
nir_band_norm = np.uint8(255 * (nir_band - nir_band.min()) / (nir_band.max() - nir_band.min()))

# Save the bands as separate images
with rasterio.open('green_band.tif', 'w', driver='GTiff', width=vv_img.shape[1], height=vv_img.shape[2], count=1, dtype=green_band_norm.dtype) as green_ds:
    green_ds.write(green_band_norm, 1)
with rasterio.open('red_band.tif', 'w', driver='GTiff', width=vv_img.shape[1], height=vv_img.shape[2], count=1, dtype=red_band_norm.dtype) as red_ds:
    red_ds.write(red_band_norm, 1)
with rasterio.open('nir_band.tif', 'w', driver='GTiff', width=vv_img.shape[1], height=vv_img.shape[2], count=1, dtype=nir_band_norm.dtype) as nir_ds:
    nir_ds.write(nir_band_norm, 1)


AttributeError: module 'rasterio' has no attribute 'features'